In [52]:
import pymysql
import random

In [48]:
# 카테고리 선택지
Category_Choices = [
    'Toners', 
    'Moisturizers', 
    'Sunscreen', 
    'Serum, Essence&Ampoules', 
    'Acne&Blemish Treatments', 
    'Exfoliating Scrubs&Peeling Gel', 
    'Eye Cream'
]

# 재료 선택지
Ingredients_Choices = [
    "UV Protection",
    "Brightening",
    "Acne-Fighting",
    "Promotes Wound Healing",
    "Anti-Aging"
]

# 피부 모델과 추천 성분 매칭
Skin_Concerns = {
    "미간주름": ["Anti-Aging"],
    "턱쳐짐": ["Anti-Aging"],
    "입술건조": ["Promotes Wound Healing"],
    "볼모공": ["Brightening", "Acne-Fighting"],
    "여드름": ["Acne-Fighting", "Promotes Wound Healing"],
    "눈가주름": ["Anti-Aging", "Brightening"],
    "이마색소침착": ["Brightening", "UV Protection"]
}

### 사용자 입력에 따른 화장품 추천 쿼리

In [47]:
conn = pymysql.connect( host="127.0.0.1", port=3306, db="cosmetic_product",
                        user="humanda", passwd="humanda")
                               
cursor = conn.cursor()

    
def recommend_products(category, ingredients, min_count):
    if not ingredients:
        return None

    placeholders = ', '.join(['%s'] * len(ingredients))

    query = f"""
    SELECT p.product_no, p.category, p.product_name, p.url
    FROM product p
    JOIN ingredient i ON p.product_no = i.product_no
    WHERE p.category = %s 
    AND i.ingredient IN ({placeholders}) 
    AND i.count >= %s
    GROUP BY p.product_no
    HAVING COUNT(DISTINCT i.ingredient) = %s;
    """

    params = [category] + ingredients + [min_count] + [len(ingredients)]
    cursor.execute(query, params)

    products = cursor.fetchall()

    # 제품 데이터가 없을 경우
    if not products:
        return None

    # 제품 리스트
    product_list = [
        {"product_no": pid, "category": cat, "product_name": name, "url": url} 
        for pid, cat, name, url in products
    ]

    recommended = random.sample(product_list, min(5, len(product_list)))

    return recommended

def user_input_simulation():
    # 카테고리 선택
    print("\n📌 선택 가능한 카테고리:")
    for index, cat in enumerate(Category_Choices, start=1):
        print(f"{index}. {cat}")

    category_input = int(input("\n원하는 카테고리 번호를 선택하세요: ")) - 1
    if 0 <= category_input < len(Category_Choices):
        category = Category_Choices[category_input]
    else:
        print("❌ 잘못된 선택입니다.")
        return
    
    # 성분 선택
    print("\n📌 선택 가능한 성분 (쉼표로 구분하여 입력하세요, 최소 1개 이상 선택 가능): ")
    for index, ing in enumerate(Ingredients_Choices, start=1):
        print(f"{index}. {ing}")

    ingredient_input = input("\n찾고 싶은 성분의 번호를 입력하세요: ")
    ingredient_indices = [int(i.strip()) - 1 for i in ingredient_input.split(",") if i.strip().isdigit()]

    if any(idx < 0 or idx >= len(Ingredients_Choices) for idx in ingredient_indices):
        print("❌ 잘못된 성분 선택입니다.")
        return

    selected_ingredients = [Ingredients_Choices[i] for i in ingredient_indices]

    # count >= 3 검색
    recommended_products = recommend_products(category, selected_ingredients, 3)

    # count >= 2 검색
    if not recommended_products:
        recommended_products = recommend_products(category, selected_ingredients, 2)

    # count >= 1 검색
    if not recommended_products:
        recommended_products = recommend_products(category, selected_ingredients, 1)

    # 해당사항 없음
    if recommended_products:
        print("\n🔹 추천 제품 리스트:")
        for index, product in enumerate(recommended_products, start=1):
            print(f"{index}. {product['product_name']} ({product['url']})")
    else:
        print("\n❌ 해당 조건에 맞는 제품이 없습니다.")

# 실행
user_input_simulation()

# 연결 종료
cursor.close()
conn.close()


📌 선택 가능한 카테고리:
1. Toners
2. Moisturizers
3. Sunscreen
4. Serum, Essence&Ampoules
5. Acne&Blemish Treatments
6. Exfoliating Scrubs&Peeling Gel
7. Eye Cream

📌 선택 가능한 성분 (쉼표로 구분하여 입력하세요, 최소 1개 이상 선택 가능): 
1. UV Protection
2. Brightening
3. Acne-Fighting
4. Promotes Wound Healing
5. Anti-Aging

🔹 추천 제품 리스트:
1. Pure Perfection Natural Sun Cream SPF50+/PA+++ (https://www.skincarisma.com/products/eyenlip/pure-perfection-natural-sun-cream-spf50-pa)
2. SOS Spot Whitening Vita Clinic Vitamin Sun Stick SPF50+ PA+++ (https://www.skincarisma.com/products/tosowoong/sos-spot-whitening-vita-clinic-vitamin-sun-stick-spf50-pa)
3. Sun Protect Face & Body Lotion SPF 50 PA (https://www.skincarisma.com/products/illuminare/sun-protect-face-body-lotion-spf-50-pa)
4. My Sunny Watery Essence SPF 50/PA+++ (https://www.skincarisma.com/products/tonymoly/my-sunny-watery-essence-spf-50-pa)
5. Tinted Sunscreen SPF50 PA++ (https://www.skincarisma.com/products/sendayu-tinggi/tinted-sunscreen-spf50-pa)


### 모델결과에 따른 화장품 추천 쿼리

In [56]:
conn = pymysql.connect( host="127.0.0.1", port=3306, db="cosmetic_product",
                        user="humanda", passwd="humanda")
                               
cursor = conn.cursor()

# 추천 제품을 가져오는 함수
def recommend_products(ingredients, min_count):
    if not ingredients:
        return None 

    placeholders = ', '.join(['%s'] * len(ingredients)) 

    query = f"""
    SELECT p.product_no, p.category, p.product_name, p.url
    FROM product p
    JOIN ingredient i ON p.product_no = i.product_no
    WHERE i.ingredient IN ({placeholders}) 
    AND i.count >= %s
    GROUP BY p.product_no
    HAVING COUNT(DISTINCT i.ingredient) = %s;
    """

    params = ingredients + [min_count] + [len(ingredients)]
    cursor.execute(query, params)

    products = cursor.fetchall()

    # 제품 데이터가 없을 경우
    if not products:
        return None

    # 제품 리스트 변환
    product_list = [
        {"product_no": pid, "category": cat, "product_name": name, "url": url}
        for pid, cat, name, url in products
    ]

    # 랜덤으로 최대 5개 추천
    recommended = random.sample(product_list, min(5, len(product_list)))

    return recommended

# 모델 예측 결과를 기반으로 제품 추천 함수
def get_recommendations(bad_conditions):
    selected_ingredients = []
    for condition in bad_conditions:
        if condition in Skin_Concerns:
            selected_ingredients.extend(Skin_Concerns[condition])

    selected_ingredients = list(set(selected_ingredients))

    print(f"\n🔍 추천할 성분 리스트: {', '.join(selected_ingredients)}")

    # count >= 3 검색
    recommended_products = recommend_products(selected_ingredients, 3)

    # count >= 2 검색
    if not recommended_products:
        recommended_products = recommend_products(selected_ingredients, 2)

    # count >= 1 검색
    if not recommended_products:
        recommended_products = recommend_products(selected_ingredients, 1)

    # 검색 실패
    if recommended_products:
        print("\n🔹 추천 제품 리스트:")
        for index, product in enumerate(recommended_products, start=1):
            print(f"{index}. {product['product_name']} ({product['url']})")
    else:
        print("\n❌ 해당 조건에 맞는 제품이 없습니다.")

# 모델에서 예측한 안 좋은 피부 입력 (예: 이마색소침착 & 여드름)
bad_conditions_from_model = ["미간주름", "여드름"]

# 제품 추천 실행
get_recommendations(bad_conditions_from_model)

# 연결 종료
cursor.close()
conn.close()


🔍 추천할 성분 리스트: Promotes Wound Healing, Acne-Fighting, Anti-Aging

🔹 추천 제품 리스트:
1. Peptide No-Sebum Repair Cream (https://www.skincarisma.com/products/d-alba-piedmont/peptide-no-sebum-repair-cream)
2. Unknown Product (https://www.skincarisma.com/products/fundamentalskin/acniclair-advanced-serum-plus)
3. Ferulic Acid + Retinol Brightening Solution (https://www.skincarisma.com/products/dr-dennis-gross-skincare/ferulic-acid-retinol-brightening-solution)
4. Sunnyproof Pink Tone Up Sun Cushion (https://www.skincarisma.com/products/peripera/sunnyproof-pink-tone-up-sun-cushion)
5. Acne Spot & Area Treatment (https://www.skincarisma.com/products/peter-thomas-roth/acne-spot-area-treatment)
